In [2]:
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram,linkage
from scipy.spatial.distance import pdist,squareform

raw_data_location = "/Users/bbrener1/taylor/raw_data/c_elegans/"

In [ ]:
# Preprocessing of single cell RNAseq data for C. elegans. 

# Source paper: Cao, Junyue, Jonathan S. Packer, Vijay Ramani, Darren A. Cusanovich, Chau Huynh, Riza Daza, Xiaojie Qiu et al. "Comprehensive single-cell transcriptional profiling of a multicellular organism." Science 357, no. 6352 (2017): 661-667.

# Url: https://science.sciencemag.org/content/357/6352/661.abstract

In [ ]:
# We first obtain the count matrix in an annoying format:

%cd {raw_data_location}

!wget ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM2599nnn/GSM2599701/suppl/GSM2599701%5FGene%2Ecount%2Ematrix%2Ecelegans%2Ecell%2ERdata%2Egz

In [ ]:
# Fuck you very much Dr. Cao, for making me install fucking R to read your bullshit
%cd {raw_data_location}
!gunzip *.gz

# Used R to convert to sparse matrix format, since matrix was totally unfiltered.

In [ ]:
# For some reason python lacks good facilities for loading COO sparse matrices, so whatever, let's do this manually 

# sparse_c_e = np.loadtxt('mtx.mtx')
rows = np.max(sparse_c_e[:,0])
columns = np.max(sparse_c_e[:,1])

In [ ]:
print(f"rows:{rows},columns:{columns}")

In [ ]:
# Note one extra row because the R mtx summary indexes from 1
from scipy.sparse import coo_matrix
sparse_c_e_np = coo_matrix((sparse_c_e[:,2],(sparse_c_e[:,0].astype(dtype=int),sparse_c_e[:,1].astype(dtype=int))),shape=(int(rows+1),int(columns+1)),dtype=float)
sparse_c_e_np.shape

In [ ]:
# Conveniently we didn't omit the header from the R row names, so it actually matches the matrix above
header = np.loadtxt('header.txt',dtype='str')

In [ ]:
# All cells have at least one read
np.sum(np.sum(sparse_c_e_np,axis=0) > 0) 

In [ ]:
# First select fatures that have at least one read
feature_mask = np.sum(sparse_c_e_np,axis=1) > 10
feature_mask = np.array(feature_mask).ravel()

In [ ]:
np.sum(feature_mask)

In [ ]:
# In order to operate on rows, we'll need to convert this to a CSR

feature_filtered = sparse_c_e_np.tocsr()[feature_mask]
filtered_header = header[feature_mask]

In [ ]:
# Now we have to examine the distribution of per-cell read sums

plt.figure()
plt.hist(np.array(np.sum(feature_filtered,axis=0)).flatten(),bins=np.arange(0,5000,100))

In [ ]:
np.sum(np.sum(feature_filtered,axis=0) > 1000)

In [ ]:
# We have a substantial number of cells showing at least 1000 UMIs (~30%), so we could simply choose these to operate on (at least for the moment)
cell_filter = np.array(np.sum(feature_filtered,axis=0) > 1000).ravel()

In [ ]:
double_filtered = feature_filtered.T[cell_filter].T

In [ ]:
double_filtered = double_filtered.T

In [ ]:
double_filtered.shape

In [ ]:
# Our matrix is still sort of porky to be operated on directly. Before we begin filtering by variance however, we should normalize by size

In [ ]:
plt.figure()
plt.hist(np.array(np.sum(double_filtered,axis=1)).flatten(),bins=50)

In [ ]:
cell_sums = np.array(np.sum(double_filtered,axis=1)).ravel()
cell_sums.shape

In [ ]:
size_corrected = (double_filtered / np.tile(cell_sums,(double_filtered.shape[1],1)).T) * 1000000


In [ ]:
plt.figure()
plt.hist(np.array(np.sum(size_corrected,axis=1)).flatten(),bins=50)
plt.show()

In [ ]:
plt.figure()
plt.hist(np.array(size_corrected).flatten(),bins=np.arange(0,1000,50),log=True)
plt.show()

In [ ]:
log_size_corrected = np.log10(1 + size_corrected)

In [ ]:
means = np.array(np.mean(log_size_corrected,axis=0)).ravel()
variances = np.array(np.var(log_size_corrected,axis=0)).ravel()

mean_sort = np.argsort(means)
var_sort = np.argsort(variances)

In [ ]:
plt.figure()
plt.title("Mean Vs Variance, Log10(n+1) TPM")
plt.scatter(means,variances,s=1)
plt.xlabel("Mean")
plt.ylabel("Variance")
plt.show()

In [ ]:
plt.figure()
plt.title("Variance by mean rank, Log10(n+1) TPM")
plt.scatter(np.arange(len(mean_sort)),variances[mean_sort],s=1,c='blue')
plt.scatter(np.arange(len(mean_sort)),means[mean_sort],s=1,c='red')
plt.xlabel("Mean")
plt.ylabel("Variance")
plt.show()

plt.figure()
plt.title("Variance by mean rank, Log10(n+1) TPM")
plt.scatter(np.arange(10000,len(mean_sort)),variances[mean_sort[10000:]],s=1,c='blue')
plt.scatter(np.arange(10000,len(mean_sort)),means[mean_sort[10000:]],s=1,c='red')
plt.xlabel("Mean")
plt.ylabel("Variance")
plt.show()

In [ ]:
plt.figure()
plt.title("Ranked Variance, Log10(n+1) TPM")
plt.scatter(np.arange(len(var_sort)),variances[var_sort],s=1)
plt.scatter(np.arange(len(var_sort)),means[var_sort],s=1)
plt.show()

In [ ]:
cov = variances/means

In [ ]:
plt.figure()
plt.title("Cov by ranked mean, Log10(n+1) TPM")
plt.scatter(np.arange(len(mean_sort[10000:])),cov[mean_sort[10000:]],s=1)
plt.show()

In [ ]:
plt.figure()
plt.title("Cov by ranked mean, Log10(n+1) TPM")
plt.scatter(np.arange(len(var_sort[10000:])),cov[var_sort[10000:]],s=1)
plt.show()

In [ ]:
# We may want to keep top 5000 genes by variance, this is a pretty diverse dataset

counts = log_size_corrected.T[var_sort[-5000:]].T
header = filtered_header[var_sort[-5000:]]


In [ ]:
counts = np.array(counts)

In [ ]:
np.savetxt(raw_data_location+"counts.tsv",counts)
np.savetxt(raw_data_location+"header.txt",header,fmt="%s")

In [3]:
counts = np.loadtxt(raw_data_location+"counts.tsv")
header = np.loadtxt(raw_data_location+"header.txt",dtype=str)

In [4]:
counts.shape

(23673, 5000)

In [5]:
import sys
sys.path.append("../src/")
import lumberjack
import tree_reader_sc as tr

In [7]:
forest = lumberjack.fit(counts,header=header,ifs=1000,ofs=1000,ss=200,dispersion_mode='ssme',sfr=0.5,norm='l2',trees=100,depth=5)

Setting context
Input:(23673, 5000)
Output:(23673, 5000)
Generating trees
Running /Users/bbrener1/taylor/rusty_lumberjack/target/release/lumberjack_1
Command: /Users/bbrener1/taylor/rusty_lumberjack/target/release/lumberjack_1 generate -ic /var/folders/_k/81hqlgss0tbf2l0t7wm4t_200000gn/T/tmpx4hz3t3z/input.counts -oc /var/folders/_k/81hqlgss0tbf2l0t7wm4t_200000gn/T/tmpx4hz3t3z/output.counts -o /var/folders/_k/81hqlgss0tbf2l0t7wm4t_200000gn/T/tmpx4hz3t3z/tmp -auto -ifh /var/folders/_k/81hqlgss0tbf2l0t7wm4t_200000gn/T/tmpx4hz3t3z/tmp.ifh -ofh /var/folders/_k/81hqlgss0tbf2l0t7wm4t_200000gn/T/tmpx4hz3t3z/tmp.ofh -ifs 1000 -ofs 1000 -ss 200 -dispersion_mode ssme -sfr 0.5 -norm l2 -trees 100 -depth 5

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
100

0 0 2.8151231087919837 0 0 0 0 0 0 0 0 0 0 0 0 0 2.8151231087919837 0 0 0 0 0 0 2.8151231087919837 0 200
300

2.19564771392417 0 2.19564771392417 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2.495291593911836 0 2.19564771392417 0 2.495291593911836 40

15500

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3.0347291316001934 0 0 15600
15700

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2.462799387353008 0 15800
15900

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 16000
16100

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3.19716035159322 16200
16300

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 16400
16500

0 0 0 0 0 0 0 0 0 0 0 2.342442568020834 0 0 0 0 0 0 0 0 2.342442568020834 0 0 0 0 16600
16700

0 0 0 0 0 0 0 0 0 0 0 0 0 2.4435785626882245 0 0 0 2.9196559717046653 0 0 0 0 0 0 0 16800
16900

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 17000
17100

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 17200
17300

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2.540205642276058 2.8406092233809788 0 0 0 17400
17500

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2.594056526088086 0 0 0 2.894533204235642 2.594056526088086 0 17600
17700

0 0 0 0 0 0 2.978440897827932 0 0 0 0 0 0 0 2.978440897827932 0 0 0 0 2.978440897827932 0 0 0 0 0 17800
1


0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 8600
8700

0 0 2.833615714666788 0 0 0 0 0 0 0 0 0 0 0 0 0 2.833615714666788 0 0 0 0 0 3.4351976609061485 0 3.532075785524405 8800
8900

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 9000
9100

0 0 2.7501088710431705 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3.2267150849127204 9200
9300

2.27460994269453 0 0 0 3.3138996594697976 0 0 0 0 0 0 0 2.7501900170332387 0 0 0 0 0 2.27460994269453 0 0 2.27460994269453 0 2.27460994269453 0 9400
9500

0 0 0 0 0 2.7355096087055735 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2.7355096087055735 0 9600
9700

0 0 0 0 0 0 0 2.4057725822676432 0 0 0 0 0 0 2.1064453725528893 0 0 0 0 0 2.4057725822676432 0 0 0 0 9800
9900

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2.978028700298033 10000
10100

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 10200
10300

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 10400
10500

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 10600
10700

0 0 0 0 0 0 0 0 0 0 0 0 0 

Initializing: 400
Initializing: 600
Initializing: 800
Initializing: 1000
Initializing: 1200
Initializing: 1400
Initializing: 1600
Initializing: 1800
Initializing: 2000
Initializing: 2200
Initializing: 2400
Initializing: 2600
Initializing: 2800
Initializing: 3000
Initializing: 3200
Initializing: 3400
Initializing: 3600
Initializing: 3800
Initializing: 4000
Initializing: 4200
Initializing: 4400
Initializing: 4600
Initializing: 4800
Made rank table with 5000 features, 23673 samples:
Initializing: 0
Initializing: 200
Initializing: 400
Initializing: 600
Initializing: 800
Initializing: 1000
Initializing: 1200
Initializing: 1400
Initializing: 1600
Initializing: 1800
Initializing: 2000
Initializing: 2200
Initializing: 2400
Initializing: 2600
Initializing: 2800
Initializing: 3000
Initializing: 3200
Initializing: 3400
Initializing: 3600
Initializing: 3800
Initializing: 4000
Initializing: 4200
Initializing: 4400
Initializing: 4600
Initializing: 4800
Made rank table with 5000 features, 23673 sampl

KeyboardInterrupt: 